<a href="https://colab.research.google.com/github/AnatoliyZhuk/Knowledge/blob/main/I_Profi_part_1/I_Profi_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Первое задание. Олимпиада Я - Профессионал <h1>

В базе данных нефтегазовых месторождений одной компании произошёл системный сбой и информация о типе месторождения по расположению (на берегу или в открытом море) была удалена. Так как эта информация крайне важна, вам необходимо восстановить её в пострадавшем датасете.

<h1>Импорт библиотек<h1>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

<h1>Работа с тренировочными данными<h1>

+ Загрузка данных
+ Посмотреть есть ли пропуски в числовых признаках
+ Посмотреть количество уникальных значений в категориальных признаках

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/AnatoliyZhuk/Knowledge/main/I_Profi_part_1/train.csv')

Для лучшего понимания необходимо описать каждую переменную, которая встречается в загруженном DataFrame.<br/>
Мы имеем 11 признаков (фичей) и 1 целевую переменную **Onshore/Oﬀshore** для предсказания.

**Tectonic regime** – тектонический режим месторождения <br/>
**Onshore/Oﬀshore** – тип месторождения по расположению <br/>
**Hydrocarbon type** – тип углеводородов <br/>
**Reservoir status** – статус разработки месторождения <br/>
**Structural setting** – структурные особенности месторождения <br/>
**Depth** – средняя глубина кровли резервуара <br/>
**Period** – геологическая система образования резервуара <br/>
**Lithology** – основной литологический состав коллектора <br/>
**Gross** – среднее значение общей толщины резервуара <br/>
**Netpay** – среднее значение эффективной углеводородонасыщенной толщины коллектора <br/>
**Porosity** – среднее значение матричной пористости <br/>
**Permeability** – среднее значение проницаемости по воздуху <br/>


Посмотрим с чем нам предстоит работать

In [3]:
data.head(3)

,Tectonic regime,Onshore/Offshore,Hydrocarbon type,Reservoir status,Structural setting,Depth,Period,Lithology,Gross,Netpay,Porosity,Permeability
0,STRIKE-SLIP/TRANSPRESSION/BASEMENT-I,OFFSHORE,OIL,DEVELOPING,INVERSION/WRENCH,3520,NEOGENE,SANDSTONE,2460.0,220.0,20.0,45.0
1,GRAVITY/EXTENSION/EVAPORITE,OFFSHORE,OIL,MATURE PRODUCTION,SALT/PASSIVE MARGIN,9967,CRETACEOUS,LIMESTONE,427.0,160.0,19.0,175.0
2,GRAVITY/EXTENSION/EVAPORITE,ONSHORE,OIL,MATURE PRODUCTION,PASSIVE MARGIN,8700,CRETACEOUS,LIMESTONE,95.0,15.0,12.0,20.0


Обозначим список названий численных признаков - **num_cols** <br/>
Обозначим список названий категориальных признаков - **cat_cols** <br/>
Обозначим список названий всех признаков, которые потребуются для модели **feature_cols** признаков <br/>
Название целевой переменной - **target_col**

In [4]:
num_cols = ['Depth', 'Gross', 'Netpay', 'Porosity', 'Permeability']
cat_cols = ['Tectonic regime', 'Hydrocarbon type', 'Reservoir status', 'Structural setting', 'Period', 'Lithology']
feature_cols = num_cols + cat_cols
target_col = 'Onshore/Offshore'

Посмотрим количество различных наблюдений в целевой переменной.

In [5]:
data['Onshore/Offshore'].value_counts()

ONSHORE             211
OFFSHORE             93
ONSHORE-OFFSHORE      5
Name: Onshore/Offshore, dtype: int64

С помощью **LabelEncoder** декодируем целевую переменную.

In [6]:
# Целевая переменная также является категориальной переменной, поэтому ее тоже нужно перевести в числовой формат
LE = LabelEncoder()
LE.fit(data['Onshore/Offshore'])
data['Onshore/Offshore'] = LE.transform(data['Onshore/Offshore'])

data.head(3)

,Tectonic regime,Onshore/Offshore,Hydrocarbon type,Reservoir status,Structural setting,Depth,Period,Lithology,Gross,Netpay,Porosity,Permeability
0,STRIKE-SLIP/TRANSPRESSION/BASEMENT-I,0,OIL,DEVELOPING,INVERSION/WRENCH,3520,NEOGENE,SANDSTONE,2460.0,220.0,20.0,45.0
1,GRAVITY/EXTENSION/EVAPORITE,0,OIL,MATURE PRODUCTION,SALT/PASSIVE MARGIN,9967,CRETACEOUS,LIMESTONE,427.0,160.0,19.0,175.0
2,GRAVITY/EXTENSION/EVAPORITE,1,OIL,MATURE PRODUCTION,PASSIVE MARGIN,8700,CRETACEOUS,LIMESTONE,95.0,15.0,12.0,20.0


Обозначим **X** все признаки (фичи), которые необходимы для обучения модели.<br/>
Произведем преобразование категориальных переменных в числовые с помощью **get.dummies**<br/>
Обозначим **y** целевую переменную

In [7]:
X = data.drop(['Onshore/Offshore'], axis=1)
X = pd.get_dummies(X, columns=cat_cols)
y = data['Onshore/Offshore']

Произведем разделение данных на тренировочную и тестовую часть.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Произведем нормализацию.

In [9]:
pca = StandardScaler()
X_train_scaled = pca.fit_transform(X_train)
X_test_scaled = pca.fit_transform(X_test)

Обучим.

In [10]:
model = LogisticRegressionCV(Cs=[100, 10, 1, 0.1, 0.01, 0.001], solver='liblinear', max_iter=10000, cv=3, scoring='accuracy', fit_intercept=True)
model.fit(X_train_scaled, y_train)

model.predict_proba(X_test_scaled)
model.score(X_test_scaled, y_test)

0.8387096774193549

Посмотрим показатели на тренировочных данных.

In [11]:
model.predict_proba(X_train_scaled)
model.score(X_train_scaled, y_train)

0.9490740740740741

In [12]:
data_test = pd.read_csv('https://raw.githubusercontent.com/AnatoliyZhuk/Knowledge/main/I_Profi_part_1/test.csv')
data_test = pd.get_dummies(data_test, columns=cat_cols)

In [13]:
data_test_end = pd.concat([data_test, X_test], join='outer', axis=0)[0:133]



In [14]:
aaa = []
for col in data_test_end.columns:
  if col not in X_test.columns:
    aaa.append(col)


In [15]:
for col in aaa:
  data_test_end.drop(col, axis=1, inplace=True)
  

In [16]:
data_test_end

,Depth,Gross,Netpay,Porosity,Permeability,Tectonic regime_COMPRESSION,Tectonic regime_COMPRESSION/EROSION,Tectonic regime_COMPRESSION/EROSION/EXTENSION/LINKED,Tectonic regime_COMPRESSION/EVAPORITE,Tectonic regime_COMPRESSION/EVAPORITE/EXTENSION/LINKED,Tectonic regime_COMPRESSION/GRAVITY/EVAPORITE/EXTENSION/LINKED,Tectonic regime_COMPRESSION/SHALE,Tectonic regime_COMPRESSION/STRIKE-SLIP/TRANSPRESSION/BASEMENT-I,Tectonic regime_EXTENSION,Tectonic regime_EXTENSION/EROSION,Tectonic regime_EXTENSION/EVAPORITE,Tectonic regime_EXTENSION/EVAPORITE/EROSION/GRAVITY,Tectonic regime_EXTENSION/EVAPORITE/INVERSION,Tectonic regime_GRAVITY/EVAPORITE/COMPRESSION,Tectonic regime_GRAVITY/EXTENSION/EVAPORITE,Tectonic regime_GRAVITY/EXTENSION/EVAPORITE/DIAPIR/SYNSEDIMENTATION,Tectonic regime_GRAVITY/EXTENSION/EVAPORITE/SYNSEDIMENTATION,Tectonic regime_GRAVITY/EXTENSION/SHALE,Tectonic regime_GRAVITY/EXTENSION/SHALE/SYNSEDIMENTATION,Tectonic regime_INVERSION/COMPRESSION/EXTENSION,Tectonic regime_INVERSION/COMPRESSION/EXTENSION/EROSION,Tectonic regime_INVERSION/COMPRESSION/EXTENSION/EVAPORITE,Tectonic regime_INVERSION/COMPRESSION/EXTENSION/SHALE,Tectonic regime_INVERSION/STRIKE-SLIP/TRANSPRESSION/EXTENSION/BASEMENT-I,Tectonic regime_STRIKE-SLIP/TRANSPRESSION,Tectonic regime_STRIKE-SLIP/TRANSPRESSION/BASEMENT-I,Hydrocarbon type_CARBON DIOXIDE,Hydrocarbon type_GAS,Hydrocarbon type_GAS-CONDENSATE,Hydrocarbon type_OIL,Reservoir status_ABANDONED,Reservoir status_CONTINUING DEVELOPMENT,Reservoir status_DECLINING PRODUCTION,Reservoir status_DEVELOPING,Reservoir status_MATURE PRODUCTION,...,Tectonic regime_TRANSTENSION/EXTENSION/SHALE/LINKED,Tectonic regime_UPLIFT/BASEMENT-I,Hydrocarbon type_METHANE HYDRATE,Reservoir status_DEPLETED,Structural setting_DELTA//FORELAND,Structural setting_DELTA/FORELAND,Structural setting_DELTA/SUB-SALT/PASSIVE MARGIN,Structural setting_DELTA/WRENCH,Structural setting_FORELAND/PASSIVE MARGIN,Structural setting_FORELAND/THRUST,Structural setting_FORELAND/WRENCH,Structural setting_INTRACRATONIC/SUB-SALT,Structural setting_INVERSION/FOREARC,Structural setting_INVERSION/SALT/RIFT,Structural setting_SALT/INVERSION/FORELAND,Structural setting_SALT/RIFT,Structural setting_SUB-SALT,Structural setting_SUB-SALT/INVERSION,Structural setting_SUB-SALT/PASSIVE MARGIN,Structural setting_THRUST/PASSIVE MARGIN,Structural setting_THRUST/SUB-THRUST/FORELAND,Structural setting_THRUST/WRENCH,Structural setting_WRENCH/DELTA,Structural setting_WRENCH/FORELAND,Structural setting_WRENCH/INVERSION/BACKARC,Structural setting_WRENCH/RIFT,Structural setting_WRENCH/THRUST,Period_ARCHEAN,Period_CAMBRIAN,Period_CARBONIFEROUS-CRETACEOUS,Period_JURASSIC-CRETACEOUS,Period_MESOZOIC,Period_PALEOZOIC,Period_PROTEROZOIC-CAMBRIAN,Lithology_BASEMENT,Lithology_CHERT,Lithology_CONGLOMERATE,Lithology_DIATOMITE,Lithology_SHALE,Lithology_VOLCANICS
0,2275,325.0,30.0,13.0,0.04,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,500,40.0,18.0,28.0,240.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5548,200.0,20.0,13.0,7.30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10100,8200.0,260.0,18.0,100.00,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8750,140.0,70.0,12.0,125.00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,

In [17]:
data_test_end = data_test_end.fillna(0)

In [18]:
data_test_scaled = pca.fit_transform(data_test_end)
data_test_scaled

array([[-1.09951352, -0.2548502 , -0.45593259, ...,  0.        ,
         0.        ,  0.        ],
       [-1.57819719, -0.42485049, -0.50006564, ...,  0.        ,
         0.        ,  0.        ],
       [-0.21684781, -0.32941173, -0.49271013, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.28042077, -0.32941173, -0.01460207, ...,  0.        ,
         0.        ,  0.        ],
       [-0.65076444, -0.20414835, -0.49271013, ...,  0.        ,
         0.        ,  0.        ],
       [-0.26188452, -0.25186774, -0.29043364, ...,  0.        ,
         0.        ,  0.        ]])

In [19]:
pred_test = model.predict(data_test_scaled)

In [20]:
pred_test

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1])

In [21]:
prediction = pd.DataFrame(pred_test, columns=['Месторождение'])
prediction

,Месторождение
0,1
1,0
2,1
3,1
4,1
...,...
128,1
129,1
130,1
131,1


In [22]:
prediction.loc[(prediction['Месторождение'] == 0), 'Месторождение'] = 'OFFSHORE'
prediction.loc[(prediction['Месторождение'] == 1), 'Месторождение'] = 'ONSHORE'
prediction

,Месторождение
0,ONSHORE
1,OFFSHORE
2,ONSHORE
3,ONSHORE
4,ONSHORE
...,...
128,ONSHORE
129,ONSHORE
130,ONSHORE
131,ONSHORE


In [23]:
prediction.to_csv('./prediction.csv.csv',header=False, index=False)

Попробуем обучить **Decision Tree** для этого задания.

In [48]:
tree = RandomForestClassifier()
parametrs = {'n_estimators' : [150, 200, 250, 30, 50, 100], 
             'max_depth' : [4, 6, 8, 10], 
             'min_samples_split' : [2, 4, 6],
             'min_samples_leaf' : [2, 5, 8, 10]}

In [49]:
grid_search_cv_clf = GridSearchCV(tree, param_grid=parametrs, cv=5)
grid_search_cv_clf.fit(X_train_scaled, y_train)
grid_search_cv_clf.best_params_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


{'max_depth': 10,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 200}

In [50]:
best_clf = grid_search_cv_clf.best_estimator_
best_clf

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [51]:
best_clf.score(X_test_scaled, y_test)

0.8172043010752689

In [52]:
pred_test_2 = best_clf.predict(data_test_scaled)

In [53]:
prediction_2 = pd.DataFrame(pred_test_2, columns=['Месторождение'])
prediction_2

,Месторождение
0,1
1,1
2,1
3,0
4,1
...,...
128,1
129,1
130,0
131,1


In [54]:
prediction_2.loc[(prediction_2['Месторождение'] == 0), 'Месторождение'] = 'OFFSHORE'
prediction_2.loc[(prediction_2['Месторождение'] == 1), 'Месторождение'] = 'ONSHORE'
prediction_2

,Месторождение
0,ONSHORE
1,ONSHORE
2,ONSHORE
3,OFFSHORE
4,ONSHORE
...,...
128,ONSHORE
129,ONSHORE
130,OFFSHORE
131,ONSHORE


In [55]:
prediction_2.to_csv('./prediction.csv',header=False, index=False)